This is just 1 plate per experiment, 3 different dates.

ABCD etc are different.

Column 1 2 3 = TS121 
Column 4 5 6 = TS163 

Column 7 8 9 = TS121 
Column 10 11 12 = TS163 

For example:

A1 A2 A3 are the same

No drugs in the plates.

Plate content (Black Flat BioGreiner Plate)

Col 1-6 -Row A - H:
0, 1E2, 1E3, 1E4, 1E5, 1E6, 1E7, 1E8 nM Arabinose

Col 7-12 - Row A - H: 
0, 1.5E4, 3E4, 6E4, 1.25E5, 2.5E5, 5E5, 1E6 nM Arabinose
 

Files
All files represent the same biological experiment

NB_OD600_mEGFP = old format 
R98 & R97 = new format that will be used in the future

In [7]:
import os
def getSubFolders(folderPath):
    subFolders = []
    for root, dirs, files in os.walk(folderPath):
        for fn in files:            
            if fn.endswith('.csv') and '_stats' in fn:
                pth = os.path.join(root, fn)
                if root in subFolders:
                    continue
                subFolders.append(pth)
    return subFolders

In [17]:
folder_input = './dataset'
lf = getSubFolders(folder_input)
import os
g = dict()
for subFolder in lf:
    print subFolder
    plate_name = os.path.split(subFolder)[1]
    g[plate_name]= subFolder

print len(g.keys())

./dataset/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92/NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92_stats.csv
./dataset/20190408 - R97 (2) - SparkControl magellan Sheet 1/20190408 - R97 (2) - SparkControl magellan Sheet 1_stats.csv
./dataset/R98 version 2 - SparkControl magellan Sheet 1/R98 version 2 - SparkControl magellan Sheet 1_stats.csv
3


In [18]:
def flatten_l(l0,l1):
    l0.extend(l1)
    return l0

def as_list(x):
    if type(x) is list:
        return x
    else:
        return [x]
    
def link_dict(dict1, dict2):    
    result = {key: flatten_l(as_list(dict1.get(key, 0)),as_list(dict2.get(key, 0)))
              for key in set(dict1) | set(dict2)}
    return result

def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

def mean_std_dict(dict0):
    import numpy as np
    result_m = {key+"_mean": np.mean(np.asarray(dict0.get(key,0))) for key in set(dict0)}
    result_std = {key+"_std": np.std(np.asarray(dict0.get(key,0))) for key in set(dict0)}
    return merge_two_dicts(result_m,result_std)

def getMeanStd(dic_growth):
    if len(dic_growth) > 1:
        dic_comb = dic_growth[0]
        for i in range(1,len(dic_growth)):                        
            dic_comb = link_dict(dic_growth[i], dic_comb)            
    else:
        print "only one value array, no mean and std"
        return dic_growth    
    result = mean_std_dict(dic_comb)
    return result



In [19]:
def getGrowthLagdGFP(stats_file):
    import csv
    m_growth = dict()
    m_lag = dict()
    m_mean_exp_dGFPdtOD = dict()
    m_mean_sta_dGFPdtOD = dict()
    m_mean_max_dGFPdtOD = dict()
    with open(stats_file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                #print "row_0",row
                line_count += 1
            else:
                #print row
                line_count += 1
                m_growth[row[0]] = float(row[1])
                m_lag[row[0]] = float(row[2])
                m_mean_exp_dGFPdtOD[row[0]] = float(row[3])
                m_mean_sta_dGFPdtOD[row[0]] = float(row[4])
                m_mean_max_dGFPdtOD[row[0]] = float(row[5])
    return m_growth, m_lag,m_mean_exp_dGFPdtOD,m_mean_sta_dGFPdtOD,m_mean_max_dGFPdtOD


In [25]:
for k_group_ex in g:
    print k_group_ex

20190408 - R97 (2) - SparkControl magellan Sheet 1_stats.csv
R98 version 2 - SparkControl magellan Sheet 1_stats.csv
NB_OD600_meGFP_480_520_FlatBlackPlates_20190205_122213 - R92_stats.csv


In [14]:
growth_rate = dict()
lag_phase = dict()

all_file = []
growth_all = []
lag_all = []
mean_exp_dGFPdtOD = []
mean_sta_dGFPdtOD = []
mean_max_dGFPdtOD = []

for k_group_ex in g:
    growth_rate[k_group_ex] = dict()
    lag_phase[k_group_ex] = dict()
    
    growth_rate[k_group_ex]['mixted_date'] = dict()
    lag_phase[k_group_ex]['mixted_date'] = dict()

    stats_file_p = g[k_group_ex]    
    growth_rate[stats_file_p]= dict()
    lag_phase[stats_file_p]= dict()        

    all_file.append(stats_file_p)
    dic_g, dic_la,m_mean_exp_dGFPdtOD,m_mean_sta_dGFPdtOD,m_mean_max_dGFPdtOD = getGrowthLagdGFP(stats_file_p)

    growth_all.append(dic_g)
    lag_all.append(dic_la)
    mean_exp_dGFPdtOD.append(m_mean_exp_dGFPdtOD)
    mean_sta_dGFPdtOD.append(m_mean_sta_dGFPdtOD)
    mean_max_dGFPdtOD.append(m_mean_max_dGFPdtOD)

        
        

In [15]:
def generateWellName():
    list1 = map(chr, range(65, 73))
    list2 = range(1,13,1)
    list2 = map(str, list2)  
    well_name = []
    for l in list1:
        for n in list2:
            well_name.append(l+n)
    return well_name

well_name = generateWellName()
print well_name

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11', 'G12', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10', 'H11', 'H12']


In [22]:
c = ['R97','R98','R92']
import pandas as pd
df = pd.DataFrame()

for idx, data in enumerate(growth_all):
    s = 'growh_rate_'+c[idx]    
    if df.empty:
        df = pd.DataFrame.from_dict(data,orient='index',columns=[s])
    else:
        df[s] = pd.Series(data)

for idx, data in enumerate(lag_all):
    s = 'lag_phase_'+c[idx]    
    df[s] = pd.Series(data)        
for idx, data in enumerate(mean_exp_dGFPdtOD):
    s = 'mean_exp_dGFPdtOD_'+c[idx]    
    df[s] = pd.Series(data)        
for idx, data in enumerate(mean_sta_dGFPdtOD):
    s = 'mean_sta_dGFPdtO_'+c[idx]    
    df[s] = pd.Series(data)        
for idx, data in enumerate(mean_max_dGFPdtOD):
    s = 'mean_max_dGFPdtOD_'+c[idx]    
    df[s] = pd.Series(data)        
        
        
df2 = df.reindex(well_name)
df2.to_csv(os.path.join(folder_input,'stats_growth_lag_gfp.csv'))
df2

,growh_rate_R97,growh_rate_R98,growh_rate_R92,lag_phase_R97,lag_phase_R98,lag_phase_R92,mean_exp_dGFPdtOD_R97,mean_exp_dGFPdtOD_R98,mean_exp_dGFPdtOD_R92,mean_sta_dGFPdtO_R97,mean_sta_dGFPdtO_R98,mean_sta_dGFPdtO_R92,mean_max_dGFPdtOD_R97,mean_max_dGFPdtOD_R98,mean_max_dGFPdtOD_R92
A1,0.000000,0.000000,0.000000,3.668045,0.930376,1.404584,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A2,0.000000,0.000000,0.000000,15.353817,0.857652,1.440954,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A3,0.000000,0.000000,0.000000,11.805340,0.997003,1.513975,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A4,0.948802,0.874393,0.833368,3.216838,3.608077,1.012132,39.147290,37.838700,424.502490,6.070272,1.415379,82.073375,131.721574,92.415510,983.742160
A5,0.979228,0.860468,0.802459,3.269217,3.564067,0.914525,39.737887,38.853945,452.027694,7.606947,1.355399,91.979333,119.710073,124.074727,914.017564
A6,0.938098,0.858414,0.785356,3.195991,3.562882,0.871889,38.954405,37.686387,429.094173,5.940852,1.297730,100.441077,141.058231,136.988256,895.457762
A7,0.000000,0.000000,0.000000,3.801767,0.000000,1.602518,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A8,0.000000,0.000000,0.000000,3.507358,0.000000,1.668580,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A9,0.000000,0.000000,0.000000,4.072225,0.000000,1.617497,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A10,0.974753,0.808150,0.824776,3.360956,3.550020,1.020720,38.407826,32.842767,388.233228,7.214398,1.413019,75.454930,116.657755,132.069502,865.850186
